In [1]:
import keras

/Users/zukako/.pyenv/versions/anaconda3-4.0.0/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

In [5]:
main_input = Input(shape=(100,), dtype='int32', name='main_input')
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)
lstm_out = LSTM(32)(x)

lstm_outを監視することでLSTMが訓練しやすくなる。

In [6]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [7]:
auxiliary_input = Input(shape=(5,), name='aux_input')
# lstm_outとaux_inputを結合したものを次の層への入力とする。
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

モデルの定義。二つの入力と二つの出力を持っている。

In [8]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

modelをコンパイルする。誤差関数などをリストで渡すとそれぞれそれに応じて評価される。<br>
optimizerやらlossを一つにした場合は共通のものになる。<br>
loss_weights罰則を定義する。

In [9]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])

In [ ]:
model.fit([headline_data, additional_data], [labels, labels],
          epochs=50, batch_size=32)

入力と出力に重み付けを行なっていれば以下のように名前で定義できる・

In [ ]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# And trained it via:
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)